In [2]:
# autofocus - big problem read into it:  
# look at how the different cell types change with the blurring 
# the distribution of confidence, but same results? plot the probabilities 
# try on the different cell groupings 
# non blur training, blur testing 

# My stuff 
# torch.save - to save the outputs of the model 
# maybe neaten code and turn into functions

# - metadata useful?
# - HOG: look at parameters 
# - RF: labels? encoding? what is my model doing?

# PCA
# reduces dimensionality by projecting to directions of max variance
# good at compact global representation
# but doesn't capture spatial strucure (e.g. edges, orientation)
# good if images are uniform size, when you want to reduce feature count drastically 
# e.g. many raw pixels to components
# good for random forest or SVM that don't like ultra high-dimension
# PCA is a statistical abstraction of the whole image, HOG is hand-crafted structure

# HOG
# captured edge direction and local shape info
# good at preserving spatial patterns (boundaries, contours)

## RF Model

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import json
import shiny_data
import pandas as pd
from PIL import Image, ImageEnhance
from skimage.feature import hog
from sklearn.decomposition import PCA
from IPython.display import display
from sklearn.metrics import classification_report

import warnings
from sklearn.exceptions import UndefinedMetricWarning
# Suppress specific warning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# 0 Immune, 1 Other, 2 Stromal, 3 Tumour
Xmat_train, Xmat_val, Xmat_tests_0, Xmat_tests_1, Xmat_tests_2, y_train_enc, y_val_enc, y_tests_enc_0, y_tests_enc_1, y_tests_enc_2 = shiny_data.load_split_images()

# reassigning to just use first one for shiny
Xmat_test = Xmat_tests_0
y_test_enc = y_tests_enc_0

# HAVE TO REDO THESE FUNCTIONS IN THE SHINY FILE
# 50
# Xmat_train, Xmat_val, Xmat_test, y_train_enc, y_val_enc, y_test_enc = shiny_data.load_split_images_50()
# 100
# Xmat_train, Xmat_val, Xmat_test, y_train_enc, y_val_enc, y_test_enc = shiny_data.load_split_images_100()

def store_results(results, y_true, y_pred, probs=None, blur=0, noise=0, hog=False, pca=False, **extra_metrics):        
    entry = {
        "blur_size": blur,
        "noise_level": noise,
        #"HOG": hog,
        #"PCA": pca, # only doing the PCA one for shiny
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, average='macro'),
        "precision": precision_score(y_true, y_pred, average='macro'),
        "recall": recall_score(y_true, y_pred, average='macro'),
    }

    f1_classes = f1_score(y_true, y_pred, average=None, zero_division=0)
    precision_classes = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_classes = recall_score(y_true, y_pred, average=None, zero_division=0)

    label_map = {
        0: "immune",
        1: "other",
        2: "stromal",
        3: "tumour"
    }
    
    for i in range(4):
        label = label_map[i]
        entry[f"f1_{label}"] = f1_classes[i]
        entry[f"precision_{label}"] = precision_classes[i]
        entry[f"recall_{label}"] = recall_classes[i]
    
    if probs is not None:
        confidences = np.max(probs, axis=1)
        entry["confidence_overall"] = np.mean(confidences)
        for cls in range(4):
            label = label_map[cls]
            cls_conf = confidences[y_pred == cls]
            entry[f"confidence_{label}_avg"] = np.mean(cls_conf) if len(cls_conf) > 0 else np.nan
            entry[f"confidence_{label}_std"] = np.std(cls_conf) if len(cls_conf) > 0 else np.nan
    
    total_preds = len(y_pred)
    for cls in range(4):
        label = label_map[cls]
        entry[f"count_pred_{label}"] = np.sum(y_pred == cls)

    # Store actual and predicted labels and the confusion matrix as strings
    entry["confusion_matrix"] = str(confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3]).tolist())

    entry.update(extra_metrics)
    results.append(entry)

def extract_hog_features(images):
    hog_features = []
    for img in images:
        if img.shape[-1] == 3:
            img = Image.fromarray((img * 255).astype(np.uint8)).convert("L")
            img = np.array(img)
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

# use shiny version (the same but cleaner), and same for hog thing if possible?
def apply_noise(images, mean=0, std=10, seed=3888):
    if std > 0:
        np.random.seed(seed)  # Consistent noise for all images in this batch
        noise = np.random.normal(mean, std, images.shape)
        noisy_images = images + noise
        noisy_images = np.clip(noisy_images, 0, 255).astype(np.uint8)
        return noisy_images
    else:
        return images

X_train_flat = Xmat_train.reshape(Xmat_train.shape[0], -1)
X_val_flat   = Xmat_val.reshape(Xmat_val.shape[0], -1)
X_test_flat  = Xmat_test.reshape(Xmat_test.shape[0], -1)

pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_flat)

# HOG not used for shiny
# X_train_hog = extract_hog_features(Xmat_train) 

# results = []
# df = pd.DataFrame(results)  # no explicit column list

[('Immune', 0), ('Other', 1), ('Stromal', 2), ('Tumour', 3)]


## PCA

In [22]:
model_pca = RandomForestClassifier(n_estimators=100, random_state=3888)
model_pca.fit(X_train_pca, y_train_enc)

# baseline
X_test_pca = pca.transform(X_test_flat)
y_pred = model_pca.predict(X_test_pca)
y_probs = model_pca.predict_proba(X_test_pca)
store_results(results, y_test_enc, y_pred, probs=y_probs)

for radius in [1, 3, 5, 7, 9, 19]: # blur
    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_flat = X_blur.reshape(X_blur.shape[0], -1)
    X_test_pca = pca.transform(X_test_flat)
    y_pred = model_pca.predict(X_test_pca)
    y_probs = model_pca.predict_proba(X_test_pca)
    store_results(results, y_test_enc, y_pred, blur=radius, probs=y_probs)

for noise_std in [1, 3, 5, 10, 20, 30]:
    seed = 3888 + noise_std  # same seed logic as baseline
    X_noisy = apply_noise(Xmat_test, std=noise_std, seed=seed)
    X_test_flat = X_noisy.reshape(X_noisy.shape[0], -1)
    X_test_pca = pca.transform(X_test_flat)
    y_pred = model_pca.predict(X_test_pca)
    y_probs = model_pca.predict_proba(X_test_pca)
    store_results(results, y_test_enc, y_pred, noise=noise_std, probs=y_probs)

for radius in [1, 3, 5, 7, 9, 19]: # blur
    for noise_std in [1, 3, 5, 10, 20, 30]:
        combo_seed = 10000 + (radius * 100) + noise_std
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = apply_noise(X_blur, std=noise_std, seed=combo_seed)
        X_test_flat = X_combo.reshape(X_combo.shape[0], -1)
        X_test_pca = pca.transform(X_test_flat)
        y_pred = model_pca.predict(X_test_pca)
        y_probs = model_pca.predict_proba(X_test_pca)
        store_results(results, y_test_enc, y_pred, blur=radius, noise=noise_std, probs=y_probs)

df = pd.DataFrame(results)
df.to_csv("rf_augmented_metrics.csv", index=False) # update to 100 here and above as needed

In [35]:
import joblib
model_pca = RandomForestClassifier(n_estimators=100, random_state=3888)
model_pca.fit(X_train_pca, y_train_enc)
joblib.dump(pca, "Base_pca.joblib")
joblib.dump(model_pca, "rf_pca_model.joblib")  # ✅ save the RF model too

['rf_pca_model.joblib']

## RF no HOG or PCA - NOT RUN/UPDATED FOR SHINY

In [ ]:
# BASELINE
model = RandomForestClassifier(n_estimators=100, random_state=3888)
model.fit(X_train_flat, y_train_enc)
y_pred = model.predict(X_test_flat)
y_probs = model.predict_proba(X_test_flat)
store_results(results, y_test_enc, y_pred, blur=0, noise=0, probs=y_probs)

# BLUR TESTS
for radius in [1, 3, 5, 7, 9, 19]:
    X_blur_test = shiny_data.apply_blur(Xmat_test, radius)
    X_test_flat = X_blur_test.reshape(X_blur_test.shape[0], -1)
    y_pred = model.predict(X_test_flat)
    y_probs = model.predict_proba(X_test_flat)
    store_results(results, y_test_enc, y_pred, blur=radius, noise=0, probs=y_probs)

# NOISE TESTS
for noise_std in [1, 3, 5, 10, 20, 30]:
    seed = 3888 + noise_std  # Ensure different but reproducible seed per level
    X_noisy_test = apply_noise(Xmat_test, std=noise_std, seed=seed)
    X_test_flat = X_noisy_test.reshape(X_noisy_test.shape[0], -1)
    y_pred = model.predict(X_test_flat)
    y_probs = model.predict_proba(X_test_flat)
    store_results(results, y_test_enc, y_pred, blur=0, noise=noise_std, probs=y_probs)

# BLUR + NOISE TESTS
for radius in [1, 3, 5, 7, 9, 19]:
    for noise_std in [1, 3, 5, 10, 20, 30]:
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        combo_seed = 10000 + (radius * 100) + noise_std  # Unique seed per combo
        X_combo = apply_noise(X_blur, std=noise_std, seed=combo_seed)
        X_test_flat = X_combo.reshape(X_combo.shape[0], -1)
        y_pred = model.predict(X_test_flat)
        y_probs = model.predict_proba(X_test_flat)
        store_results(results, y_test_enc, y_pred, blur=radius, noise=noise_std, probs=y_probs)

# Show progress
#df = pd.DataFrame(results)
#print("\n=== After Blur and Noise Augmentations ===")
#cols = [
#    "HOG", "PCA", "Blur", "Noise",
#    "accuracy", "f1", "precision", "recall",
#    "f1_class_0", "f1_class_1", "f1_class_2", "f1_class_3",
#    "precision_class_0", "precision_class_1", "precision_class_2", "precision_class_3",
#    "recall_class_0", "recall_class_1", "recall_class_2", "recall_class_3",
#    "avg_conf", "avg_conf_class_0", "avg_conf_class_1", "avg_conf_class_2", "avg_conf_class_3",
#    "count_pred_class_0", "count_pred_class_1", "count_pred_class_2", "count_pred_class_3"
#]

#display(df[cols].sort_values(by=["PCA", "Blur", "Noise"]).reset_index(drop=True))

## HOG - NOT RUN/UPDATED FOR SHINY

In [ ]:
# === HOG MODEL ===
model_hog = RandomForestClassifier(n_estimators=100, random_state=3888)
model_hog.fit(X_train_hog, y_train_enc)

# BASELINE HOG TEST
X_test_hog = extract_hog_features(Xmat_test)
y_pred = model_hog.predict(X_test_hog)
probs = model_hog.predict_proba(X_test_hog)
store_results(results, y_test_enc, y_pred, probs=probs, hog=True)

# BLUR + HOG
for radius in [1, 3, 5, 7, 9, 19]:
    X_blur = shiny_data.apply_blur(Xmat_test, radius)
    X_test_hog = extract_hog_features(X_blur)
    y_pred = model_hog.predict(X_test_hog)
    probs = model_hog.predict_proba(X_test_hog)
    store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, hog=True)

# NOISE + HOG
for noise_std in [1, 3, 5, 10, 20, 30]:
    seed = 3888 + noise_std  # ensures consistent noise for this std level
    X_noisy = apply_noise(Xmat_test, std=noise_std, seed=seed)
    X_test_hog = extract_hog_features(X_noisy)
    y_pred = model_hog.predict(X_test_hog)
    probs = model_hog.predict_proba(X_test_hog)
    store_results(results, y_test_enc, y_pred, probs=probs, noise=noise_std, hog=True)

# BLUR + NOISE + HOG
for radius in [1, 3, 5, 7, 9, 19]:
    for noise_std in [1, 3, 5, 10, 20, 30]:
        combo_seed = 10000 + (radius * 100) + noise_std
        X_blur = shiny_data.apply_blur(Xmat_test, radius)
        X_combo = apply_noise(X_blur, std=noise_std, seed=combo_seed)
        X_test_hog = extract_hog_features(X_combo)
        y_pred = model_hog.predict(X_test_hog)
        probs = model_hog.predict_proba(X_test_hog)
        store_results(results, y_test_enc, y_pred, probs=probs, blur=radius, noise=noise_std, hog=True)

# Show progress
#df = pd.DataFrame(results)
#print("\n=== After Blur + Noise + HOG Augmentations ===")
#cols = [
#    "HOG", "PCA", "Blur", "Noise",
#    "accuracy", "f1", "precision", "recall",
#    "f1_class_0", "f1_class_1", "f1_class_2", "f1_class_3",
#    "precision_class_0", "precision_class_1", "precision_class_2", "precision_class_3",
#    "recall_class_0", "recall_class_1", "recall_class_2", "recall_class_3",
#    "avg_conf", "avg_conf_class_0", "avg_conf_class_1", "avg_conf_class_2", "avg_conf_class_3",
#    "count_pred_class_0", "count_pred_class_1", "count_pred_class_2", "count_pred_class_3"
#]

#display(df[cols].sort_values(by=["PCA", "Blur", "Noise"]).reset_index(drop=True))

In [ ]:
df.to_csv("RF_results_100.csv", index=False) # update to 100 here and above as needed